<a href="https://colab.research.google.com/github/jimwellh/NYCU-Introduction-to-Data-science/blob/main/0811033_%E9%BB%83%E5%8B%81%E7%91%8B_hw03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Introduction to Data Science** Homework 3
0811033_黃勁瑋_hw02\
2023/10/9


*   Use R, Python, and suitable computer packages to analyze the data set with missing data
(NA) that you select.
*   Try to solve the issues of missing values and the other types of data
impurities.
*   Use Different Types of Data Imputation Methods for handling missing data in your
datasets.
*   Explain the results you obtain
*   Discuss possible problems you plan to investigate for future studies



In this homework, the same "Electric Power Consumption" dataset from the previous homework is used. The data originally has a Kaggle uability score of 10.00. However, to complete this homework, I manually delete some values in the dataset and try to fill out the blank values using several different methods. The result is then compared with the original value to discuss which data cleanning method is more sutiable for this data.  

**Import data and necessary libaries**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import pandas_profiling
import seaborn as sns
import os
data = pd.read_csv('/content/drive/MyDrive/Introduction to Data science/Data sets/powerconsumption.csv')
#check is the data is read successfully
print(data.head(10))
print(data.shape)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
        Datetime  Temperature  Humidity  WindSpeed  GeneralDiffuseFlows  \
0  1/1/2017 0:00        6.559      73.8      0.083                0.051   
1  1/1/2017 0:10        6.414      74.5      0.083                0.070   
2  1/1/2017 0:20        6.313      74.5      0.080                0.062   
3  1/1/2017 0:30        6.121      75.0      0.083                0.091   
4  1/1/2017 0:40        5.921      75.7      0.081                0.048   
5  1/1/2017 0:50        5.853      76.9      0.081                0.059   
6  1/1/2017 1:00        5.641      77.7      0.080                0.048   
7  1/1/2017 1:10        5.496      78.2      0.085                0.055   
8  1/1/2017 1:20        5.678      78.1      0.081                0.066   
9  1/1/2017 1:30        5.491      77.3      0.082                0.062   

   DiffuseFlows  PowerConsumption_Zone1  Powe

**Check if exist outlier**

The variables in that datasets have physical meanings, so using physical limits to check outliers may be a good approach for this homework.

In [ ]:
data.describe()

,Temperature,Humidity,WindSpeed,GeneralDiffuseFlows,DiffuseFlows,PowerConsumption_Zone1,PowerConsumption_Zone2,PowerConsumption_Zone3
count,52416.000000,52416.000000,52416.000000,52416.000000,52416.000000,52416.000000,52416.000000,52416.000000
mean,18.810024,68.259518,1.959489,182.696614,75.028022,32344.970564,21042.509082,17835.406218
std,5.815476,15.551177,2.348862,264.400960,124.210949,7130.562564,5201.465892,6622.165099
min,3.247000,11.340000,0.050000,0.004000,0.011000,13895.696200,8560.081466,5935.174070
25%,14.410000,58.310000,0.078000,0.062000,0.122000,26310.668692,16980.766032,13129.326630
50%,18.780000,69.860000,0.086000,5.035500,4.456000,32265.920340,20823.168405,16415.117470
75%,22.890000,81.400000,4.915000,319.600000,101.000000,37309.018185,24713.717520,21624.100420
max,40.010000,94.800000,6.483000,1163.000000,936.000000,52204.395120,37408.860760,47598.326360


Using the descirbe function we can find the min and max value of each variables. Unfortunately, the data provider did not provide any description about the unit of each variable. From the value range I guess that Temperature is in degree Celsius, WindSpeed is in m/s and Humidity is actuall relative humidity with no unit and value range from 0% to 100%. However, the infomation is not enough to help determine if any value is an outlier in this data set.  

The IQR method is another way that could be used to find if there exist outliers which defines outliers as values greater then Q3(75%)+1.5*IQR and less then Q1(25%)-1.5*IQR.

In [ ]:
def find_outliers_iqr(column):
  Q1 = column.quantile(0.25)
  Q3 = column.quantile(0.75)
  IQR = Q3 - Q1
  lower_bound = Q1 - 1.5 * IQR
  upper_bound = Q3 + 1.5 * IQR
  outliers = column[(column < lower_bound) | (column > upper_bound)]
  return outliers

outlier_Temp = find_outliers_iqr(data["Temperature"])
outlier_Humid = find_outliers_iqr(data["Humidity"])
outlier_Wind = find_outliers_iqr(data["WindSpeed"])
outlier_Generaldiff = find_outliers_iqr(data["GeneralDiffuseFlows"])
outlier_Diff = find_outliers_iqr(data["DiffuseFlows"])
outlier_Power1 = find_outliers_iqr(data["PowerConsumption_Zone1"])
outlier_Power2 = find_outliers_iqr(data["PowerConsumption_Zone2"])
outlier_Power3 = find_outliers_iqr(data["PowerConsumption_Zone3"])


print("The outliers in column Temperature is","\n",outlier_Temp,"\n","The number of outliers occupy",outlier_Temp.shape[0]/data.shape[0]*100,"% of the data size","\n",)
print("The outliers in column Humidity is","\n",outlier_Humid,"\n","The number of outliers occupy",outlier_Humid.shape[0]/data.shape[0]*100,"% of the data size","\n",)
print("The outliers in column WindSpeed is","\n",outlier_Wind,"\n","The number of outliers occupy",outlier_Wind.shape[0]/data.shape[0]*100,"% of the data size","\n",)
print("The outliers in column GeneralDiffuseFlows is","\n",outlier_Generaldiff,"\n","The number of outliers occupy",outlier_Generaldiff.shape[0]/data.shape[0]*100,"% of the data size","\n",)
print("The outliers in column DiffuseFlows is","\n",outlier_Diff,"\n","The number of outliers occupy",outlier_Diff.shape[0]/data.shape[0]*100,"% of the data size","\n",)
print("The outliers in column PowerConsumption_Zone1 is","\n",outlier_Power1,"\n","The number of outliers occupy",outlier_Power1.shape[0]/data.shape[0]*100,"% of the data size","\n",)
print("The outliers in column PowerConsumption_Zone2 is","\n",outlier_Power2,"\n","The number of outliers occupy",outlier_Power2.shape[0]/data.shape[0]*100,"% of the data size","\n",)
print("The outliers in column PowerConsumption_Zone3 is","\n",outlier_Power3,"\n","The number of outliers occupy",outlier_Power3.shape[0]/data.shape[0]*100,"% of the data size","\n",)

The outliers in column Temperature is 
 27000    35.78
27001    36.21
27002    36.37
27003    36.69
27004    36.59
         ...  
31910    35.81
32056    35.94
32057    35.89
32058    35.66
32059    35.64
Name: Temperature, Length: 142, dtype: float64 
 The number of outliers occupy 0.2709096459096459 % of the data size 

The outliers in column Humidity is 
 17968    23.52
17969    23.66
19088    21.96
19089    20.92
19090    20.62
         ...  
38116    14.80
38117    19.21
38118    20.21
38119    21.64
38120    22.61
Name: Humidity, Length: 291, dtype: float64 
 The number of outliers occupy 0.5551739926739927 % of the data size 

The outliers in column WindSpeed is 
 Series([], Name: WindSpeed, dtype: float64) 
 The number of outliers occupy 0.0 % of the data size 

The outliers in column GeneralDiffuseFlows is 
 6411     803.0
7422     810.0
7423     877.0
7424     845.0
7425     818.0
         ...  
39539    819.0
40685    818.0
40971    822.0
43999    841.0
44000    837.0
Name: 

The above code shows the outlier of each column. According to the definition, there is no outlier in WindSpeed and PowerConsumption_Zone1. For Temperature, Humidity, and PowerConsumption_Zone2 the IQR method seems to perform fine where less then 1% of data are labeled as outliers. However, for GeneralDiffuseFlows, DiffuseFlows, and PowerConsumption_Zone3 over 2% or more data are labeled as outliers. In the worst case, more then 8% of DiffuseFlows data are labeled as outliers using IQR method. That is, for these 3 variables, using IQR method to detect outliers may not be a good way.      

Another way to determine outliers is to see if there is any studden impulse in the data since based on the physical properties of each variables, the value should change continuously and smoothly. Define a "sudden change" as:  (X_current-X_previous)/X_current>0.5.

In [ ]:
def find_outliers_smooth(column):
  outliers=pd.Series([], dtype=column.dtype)
  previous_value = None
  for value in column:
    if previous_value is not None and (value - previous_value)/value>0.5:
      outliers=pd.concat([outliers, pd.Series(value)])
    previous_value = value
  return outliers

print("The outliers in Temperature is:","\n",find_outliers_smooth(data["Temperature"]),"\n")
print("The outliers in Humidity is:","\n",find_outliers_smooth(data["Humidity"]),"\n")
print("The outliers in WindSpeed is:","\n",find_outliers_smooth(data["WindSpeed"]),"\n")
print("The outliers in GeneralDiffuseFlows is:","\n",find_outliers_smooth(data["GeneralDiffuseFlows"]),"\n")
print("The outliers in DiffuseFlows is:","\n",find_outliers_smooth(data["DiffuseFlows"]),"\n")
print("The outliers in PowerConsumption_Zone1 is:","\n",find_outliers_smooth(data["PowerConsumption_Zone1"]),"\n")
print("The outliers in PowerConsumption_Zone2 is:","\n",find_outliers_smooth(data["PowerConsumption_Zone2"]),"\n")
print("The outliers in PowerConsumption_Zone3 is:","\n",find_outliers_smooth(data["PowerConsumption_Zone3"]),"\n")

The outliers in Temperature is: 
 Series([], dtype: float64) 

The outliers in Humidity is: 
 Series([], dtype: float64) 

The outliers in WindSpeed is: 
 0    2.660
0    2.287
0    4.708
0    0.295
0    2.981
     ...  
0    4.917
0    2.766
0    2.442
0    2.853
0    3.908
Length: 106, dtype: float64 

The outliers in GeneralDiffuseFlows is: 
 0     0.099
0     0.187
0     0.955
0     3.188
0     6.643
      ...  
0     3.126
0     6.969
0    14.120
0    52.900
0     0.048
Length: 2586, dtype: float64 

The outliers in DiffuseFlows is: 
 0    0.949
0    3.134
0    6.494
0    0.890
0    2.867
     ...  
0    6.638
0    0.111
0    0.968
0    3.060
0    6.765
Length: 1866, dtype: float64 

The outliers in PowerConsumption_Zone1 is: 
 Series([], dtype: float64) 

The outliers in PowerConsumption_Zone2 is: 
 Series([], dtype: float64) 

The outliers in PowerConsumption_Zone3 is: 
 Series([], dtype: float64) 



From the code above we find that there is no outliers in Temperature, Humidity, and Zone 1, 2 and 3 Power Consumption. Moreover, since the time window is ten minutes in the data set and wind condition can certainly fluctuate under a high frequency in 10 minutes,  the outlier detected using this function on Wind Speed, General Diffuse Flows, and Diffuse Flows is not accurate.


So far we had introduced 3 ways to find the outliers in this dataset. Since this dataste records real life physical condition, using the IQR method may be less proper then physical limit and smooth method. Futhermore, all 3 method has its limitarion and cannot detect the outliers in variable GeneralDiffuseFlows and DiffuseFlows. Therefore, I decide to keep the original data without any modification for further analysis.  

**Manually make the data "dirty"**

In [ ]:
data_original=data.copy()
data.iloc[[99,350,1314],1]=np.nan
data.iloc[[200,2370,22214],2]=np.nan
data.iloc[[9999,34450,46314],3]=np.nan
data.iloc[[43453,31250,13714],4]=np.nan
data.iloc[[3399,20350,21314],5]=np.nan
data.iloc[[7,4444,9856],6]=np.nan
data_modified=data
print(data_modified.isnull().any(),"\n")
print(data_original.isnull().any())

Datetime                  False
Temperature                True
Humidity                   True
WindSpeed                  True
GeneralDiffuseFlows        True
DiffuseFlows               True
PowerConsumption_Zone1     True
PowerConsumption_Zone2    False
PowerConsumption_Zone3    False
dtype: bool 

Datetime                  False
Temperature               False
Humidity                  False
WindSpeed                 False
GeneralDiffuseFlows       False
DiffuseFlows              False
PowerConsumption_Zone1    False
PowerConsumption_Zone2    False
PowerConsumption_Zone3    False
dtype: bool


**Handling missing data**

First, try to handle missing data by replacing them with the mean value of the entire dataset  (ignoring missing values).

In [ ]:
data_allMean=data.copy()
data_allMean.loc[data_modified["Temperature"].isna(),"Temperature"]=data_modified["Temperature"].mean()
data_allMean.loc[data_modified["Humidity"].isna(),"Humidity"]=data_modified["Humidity"].mean()
data_allMean.loc[data_modified["WindSpeed"].isna(),"WindSpeed"]=data_modified["WindSpeed"].mean()
data_allMean.loc[data_modified["GeneralDiffuseFlows"].isna(),"GeneralDiffuseFlows"]=data_modified["GeneralDiffuseFlows"].mean()
data_allMean.loc[data_modified["DiffuseFlows"].isna(),"DiffuseFlows"]=data_modified["DiffuseFlows"].mean()
data_allMean.loc[data_modified["PowerConsumption_Zone1"].isna(),"PowerConsumption_Zone1"]=data_modified["PowerConsumption_Zone1"].mean()

Next, we could also try replacing the missing data with median

In [ ]:
data_median=data.copy()
data_median.loc[data_modified["Temperature"].isna(),"Temperature"]=data_modified["Temperature"].quantile(0.5)
data_median.loc[data_modified["Humidity"].isna(),"Humidity"]=data_modified["Humidity"].quantile(0.5)
data_median.loc[data_modified["WindSpeed"].isna(),"WindSpeed"]=data_modified["WindSpeed"].quantile(0.5)
data_median.loc[data_modified["GeneralDiffuseFlows"].isna(),"GeneralDiffuseFlows"]=data_modified["GeneralDiffuseFlows"].quantile(0.5)
data_median.loc[data_modified["DiffuseFlows"].isna(),"DiffuseFlows"]=data_modified["DiffuseFlows"].quantile(0.5)
data_median.loc[data_modified["PowerConsumption_Zone1"].isna(),"PowerConsumption_Zone1"]=data_modified["PowerConsumption_Zone1"].quantile(0.5)

Since the values should be smooth an continuous, we could try linear interpolation to fill the data.

In [ ]:
data_linear=data.copy()
data_linear= data_modified.interpolate()

**Calculate and compare the accuracy of each method.**

Define the accuracy of the predicted missing data as below

In [ ]:
def accuracy(mod_data,org_data,data,feature):
  return (mod_data.loc[data[feature].isna(),feature]-org_data.loc[data[feature].isna(),feature])/org_data.loc[data[feature].isna(),feature]

The accuracy of mean method is:

In [ ]:
print("The accuracy using the mean of the enire data in Temperature is:","\n",accuracy(data_allMean,data_original,data_modified,"Temperature"),"\n")
print("The accuracy using the mean of the enire data in Humidity is:","\n",accuracy(data_allMean,data_original,data_modified,"Humidity"),"\n")
print("The accuracy using the mean of the enire data in WindSpeed is:","\n",accuracy(data_allMean,data_original,data_modified,"WindSpeed"),"\n")
print("The accuracy using the mean of the enire data in GeneralDiffuseFlows is:","\n",accuracy(data_allMean,data_original,data_modified,"GeneralDiffuseFlows"),"\n")
print("The accuracy using the mean of the enire data in DiffuseFlows is:","\n",accuracy(data_allMean,data_original,data_modified,"DiffuseFlows"),"\n")
print("The accuracy using the mean of the enire data in PowerConsumption_Zone1 is:","\n",accuracy(data_allMean,data_original,data_modified,"PowerConsumption_Zone1"),"\n")

The accuracy using the mean of the enire data in Temperature is: 
 99      0.211222
350     0.405850
1314    0.286612
Name: Temperature, dtype: float64 

The accuracy using the mean of the enire data in Humidity is: 
 200     -0.212700
2370     0.106304
22214   -0.208133
Name: Humidity, dtype: float64 

The accuracy using the mean of the enire data in WindSpeed is: 
 9999    -0.602088
34450   -0.601440
46314   -0.601683
Name: WindSpeed, dtype: float64 

The accuracy using the mean of the enire data in GeneralDiffuseFlows is: 
 13714    6299.234720
31250    2646.924737
43453      12.297439
Name: GeneralDiffuseFlows, dtype: float64 

The accuracy using the mean of the enire data in DiffuseFlows is: 
 3399      -0.874175
20350     -0.663149
21314    720.314331
Name: DiffuseFlows, dtype: float64 

The accuracy using the mean of the enire data in PowerConsumption_Zone1 is: 
 7       0.271116
4444   -0.250115
9856    0.030556
Name: PowerConsumption_Zone1, dtype: float64 



The accuracy of median method is:

In [ ]:
print("The accuracy using the median of the enire data in Temperature is:","\n",accuracy(data_median,data_original,data_modified,"Temperature"),"\n")
print("The accuracy using the median of the enire data in Humidity is:","\n",accuracy(data_median,data_original,data_modified,"Humidity"),"\n")
print("The accuracy using the median of the enire data in WindSpeed is:","\n",accuracy(data_median,data_original,data_modified,"WindSpeed"),"\n")
print("The accuracy using the median of the enire data in GeneralDiffuseFlows is:","\n",accuracy(data_median,data_original,data_modified,"GeneralDiffuseFlows"),"\n")
print("The accuracy using the median of the enire data in DiffuseFlows is:","\n",accuracy(data_median,data_original,data_modified,"DiffuseFlows"),"\n")
print("The accuracy using the median of the enire data in PowerConsumption_Zone1 is:","\n",accuracy(data_median,data_original,data_modified,"PowerConsumption_Zone1"),"\n")

The accuracy using the median of the enire data in Temperature is: 
 99      0.209272
350     0.403587
1314    0.284542
Name: Temperature, dtype: float64 

The accuracy using the median of the enire data in Humidity is: 
 200     -0.194233
2370     0.132253
22214   -0.189559
Name: Humidity, dtype: float64 

The accuracy using the median of the enire data in WindSpeed is: 
 9999    -0.982535
34450   -0.982506
46314   -0.982517
Name: WindSpeed, dtype: float64 

The accuracy using the median of the enire data in GeneralDiffuseFlows is: 
 13714    172.655172
31250     71.985507
43453     -0.633479
Name: GeneralDiffuseFlows, dtype: float64 

The accuracy using the median of the enire data in DiffuseFlows is: 
 3399     -0.992531
20350    -0.980004
21314    41.817308
Name: DiffuseFlows, dtype: float64 

The accuracy using the median of the enire data in PowerConsumption_Zone1 is: 
 7       0.268012
4444   -0.251946
9856    0.028040
Name: PowerConsumption_Zone1, dtype: float64 



The accuracy of linear interpolate method is:

In [ ]:
print("The accuracy using the linear interpolate in Temperature is:","\n",accuracy(data_linear,data_original,data_modified,"Temperature"),"\n")
print("The accuracy using the linear interpolate in Humidity is:","\n",accuracy(data_linear,data_original,data_modified,"Humidity"),"\n")
print("The accuracy using the linear interpolate in WindSpeed is:","\n",accuracy(data_linear,data_original,data_modified,"WindSpeed"),"\n")
print("The accuracy using the linear interpolatea in GeneralDiffuseFlows is:","\n",accuracy(data_linear,data_original,data_modified,"GeneralDiffuseFlows"),"\n")
print("The accuracy using the linear interpolate in DiffuseFlows is:","\n",accuracy(data_linear,data_original,data_modified,"DiffuseFlows"),"\n")
print("The accuracy using the linear interpolate in PowerConsumption_Zone1 is:","\n",accuracy(data_linear,data_original,data_modified,"PowerConsumption_Zone1"),"\n")

The accuracy using the linear interpolate in Temperature is: 
 99     -0.001288
350    -0.003737
1314   -0.001710
Name: Temperature, dtype: float64 

The accuracy using the linear interpolate in Humidity is: 
 200      0.001153
2370     0.026904
22214   -0.002320
Name: Humidity, dtype: float64 

The accuracy using the linear interpolate in WindSpeed is: 
 9999    -2.030869e-04
34450    3.051261e-04
46314    1.805608e-16
Name: WindSpeed, dtype: float64 

The accuracy using the linear interpolatea in GeneralDiffuseFlows is: 
 13714    0.758621
31250    0.065217
43453    0.442831
Name: GeneralDiffuseFlows, dtype: float64 

The accuracy using the linear interpolate in DiffuseFlows is: 
 3399    -0.003858
20350   -0.094522
21314    0.302885
Name: DiffuseFlows, dtype: float64 

The accuracy using the linear interpolate in PowerConsumption_Zone1 is: 
 7      -0.002268
4444   -0.002465
9856   -0.001171
Name: PowerConsumption_Zone1, dtype: float64 



As a result we can see that using linear interpolation to predict missing values is better that replacing them by the mean or median value. The statement is true for all varaiables include Temperature, Humidity, and so on.

**Conclusion**

In this homework we've discussed using 3 ways to find the outliers in our data. We show the pros and cons of each method and in the end descide to keep the data as it was for future analysis. All 3 method we tried has its limitation and other methods could be tried and investigated for future studies.


On top of that, we manually erase some data points in our data and tried replacing them by mean, median and linear interpolation. In short we discover that linear interpolation is the best way among all 3 methods to deal with missing values in this dataset wheather which varaible we are trying to replace. For future studies of filling out missing values we could try implementing moving average method, or sine the data set is a time serie data across, we could try minning the historical weather data around that specific time where the value is missing on the internet and replace it. The method requires data minning skills but should be the most accurate.     